In [ ]:
# .env 파일내용
# OPENAI_API_KEY="TEST KEY"

# 실제키가 들어있는 위치
# https://buly.kr/GvnaAF3

In [ ]:
%pip install langchain langchain-openai python-dotenv

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.getenv("OPENAI_API_KEY")[:5]

'sk-pr'

In [ ]:
#  Langchain 의 핵심 컴포넌트
# PromptTemplate : LLM에 전달할 프롬프트를 생성
# ChatOpenAI : OpenAI의 챗 모델 - GPT-5, GPT-4, GPT-3.5 등
# Runnable : 공통 인터페이스  .invoke()
# StrOutputParser : 문자열 출력 파서

In [11]:
from langchain_core.prompts import PromptTemplate
template = '{sample} 과 어울리는 회사이름 추천해줘?'
prompt = PromptTemplate.from_template(template)
formatted_prompt =  prompt.format(sample='coffe')
print(formatted_prompt)

coffe 과 어울리는 회사이름 추천해줘?


In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)
response = llm.invoke( ["human",'Hello, how are you?'] )
print(response)

content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 18, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C7udoJNNONcUhDLIsxGwoLrGac6nV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--76b2e806-80c0-4464-9089-c90d15ff9260-0' usage_metadata={'input_tokens': 18, 'output_tokens': 29, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_text = parser.invoke(response)
print(parsed_text)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [14]:
# 체인
chain = prompt | llm | parser
result = chain.invoke({'sample':'coffee'})
print(f'result = {result}')

result = 커피와 어울리는 회사 이름 몇 가지를 추천해드릴게요:

1. **커피하우스** - 아늑한 분위기를 연상시키는 이름
2. **커피정원** - 자연과 함께하는 느낌을 주는 이름
3. **커피공방** - 수제 커피의 느낌을 강조하는 이름
4. **아침의 커피** - 하루를 시작하는 커피의 중요성을 담은 이름
5. **커피와 이야기** - 사람들과의 소통을 강조하는 이름
6. **커피의 향기** - 커피의 매력을 부각시키는 이름
7. **커피마을** - 친근하고 따뜻한 느낌을 주는 이름
8. **커피여행** - 다양한 커피의 세계를 탐험하는 느낌을 주는 이름
9. **커피와 함께** - 일상 속에서 커피의 동반자 역할을 강조하는 이름
10. **커피의 순간** - 특별한 순간을 함께하는 커피를 의미하는 이름

이 중에서 마음에 드는 이름이 있으면 좋겠네요!


In [15]:
# 과금정보 추적
llm = ChatOpenAI(model = 'gpt-4o-mini',temperature=0.7)
prompt = 'langchain 에 대해 한 문장으로 설명해줘'
result = llm.invoke(prompt)
print(f'질문 : {prompt}')
print(f'답변 : {result.content}')

질문 : langchain 에 대해 한 문장으로 설명해줘
답변 : LangChain은 자연어 처리(NLP) 모델을 활용하여 다양한 애플리케이션을 구축하고 연결할 수 있도록 지원하는 프레임워크입니다.


In [16]:
print(f'토큰 사용 정보 : {result.usage_metadata}')

토큰 사용 정보 : {'input_tokens': 18, 'output_tokens': 34, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
# In-Memory 캐시(메모리 캐시) 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# 인메모리캐시 설정
set_llm_cache(InMemoryCache()) # 캐시 활성

In [1]:
# 데이터베이스 방식 캐시
from pathlib import Path
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

# 현재 작업 디렉토리 기준으로 절대 경로 생성
db_path = Path().resolve() / "langchain.db"
print(db_path)
set_llm_cache(SQLiteCache(database_path=str(db_path)))

D:\AI\langchain.db


In [2]:
import time
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)
query = '우울할때 먹는 음식하나 추천 해 줘'

In [5]:
start = time.time()
# 첫번째 호출  캐시에 없으면 api 호출
result1 = llm.invoke(query)
end = time.time()
print(f'응답1: {result1.content} {end-start:.6f} 초')

응답1: 우울할 때는 기분을 좋게 해주는 음식이 도움이 될 수 있습니다. 그 중 하나로 **다크 초콜릿**을 추천합니다. 다크 초콜릿은 기분을 좋게 해주는 화학물질인 세로토닌과 엔도르핀의 분비를 촉진하는 데 도움을 줄 수 있습니다. 또한, 항산화 물질이 풍부하여 스트레스 해소에도 좋습니다. 적당량을 즐기면서 기분을 전환해 보세요! 0.001859 초


In [4]:
# 두번째 호출  동일한  query 호출  api 미 호출
start = time.time()
result2 = llm.invoke(query)
end = time.time()
print(f'응답1: {result2.content} {end-start:.6f} 초')

응답1: 우울할 때는 기분을 좋게 해주는 음식이 도움이 될 수 있습니다. 그 중 하나로 **다크 초콜릿**을 추천합니다. 다크 초콜릿은 기분을 좋게 해주는 화학물질인 세로토닌과 엔도르핀의 분비를 촉진하는 데 도움을 줄 수 있습니다. 또한, 항산화 물질이 풍부하여 스트레스 해소에도 좋습니다. 적당량을 즐기면서 기분을 전환해 보세요! 0.001648 초


In [ ]:
# SQLite 방식 캐시
%pip install langchain_community